# Rotating Neutron Star Sequences

Visualize properties of RNS sequences

Setup notebook

In [ ]:
from itertools import cycle
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata, interp1d
from scipy.optimize import brentq
from itertools import chain

In [ ]:
%matplotlib

color_list = mpl.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(color_list)

In [ ]:
class Struct(object):
    pass

Options

In [ ]:
options = Struct()
options.nseq = 20

## Read the data

In [ ]:
names = ['ratio', 'rho_c', 'M', 'M_0', 'r_star', 'Omega', 'Omega_K', 'I_45', 'a']
df = pd.read_csv('data/output.dat', names = names, sep = ' ').sort_values(['rho_c', 'M']) # Data Frame
df['diff_Omega'] = np.abs(df['Omega_K'] - df['Omega'])
df["J"] = df["a"]*df["M"]**2
df['a'] = np.nan_to_num(df['J'] / (df['M'] ** 2)) # Handles 0/0 => 0
df = df[df.rho_c > 0.5]
rho_c = np.array(sorted(list(df["rho_c"].unique())))

## Analysis

Find the mass shedding curve

In [ ]:
L = []
for rho in rho_c:
    sel = df[df.rho_c == rho]
    idx = np.ma.masked_invalid(sel["diff_Omega"]).argmin()
    L.append(sel.iloc[idx])
mass_shedding = pd.DataFrame(L).sort_values(["rho_c", "M"])

imax = np.ma.masked_invalid(mass_shedding["M"]).argmax()
rho_K_max = mass_shedding["rho_c"].iloc[imax]
M_K_max = mass_shedding["M"].iloc[imax]

Construct sequences of constant rho_c

In [ ]:
fig, ax = plt.subplots()

for rho in rho_c[::10]:
    sel = df[df.rho_c == rho]
    ax.plot(sel["M"], sel["J"])
ax.set_xlabel("$M$")
ax.set_ylabel("$J$")

Construct sequences of constant J

In [ ]:
Jseq = np.arange(0, df["J"].max(), df["J"].max()/options.nseq)
Mseq = [[] for i in range(options.nseq)]
for rho in rho_c:
    sel = df[df.rho_c == rho]
    for i in range(options.nseq):
        Mseq[i].append(np.interp(Jseq[i], sel["J"], sel["M"], left=np.nan, right=np.nan))

rho_c_max, M_max = [], []
for i in range(options.nseq):
    Mseq[i] = np.array(Mseq[i])
    imax = np.ma.masked_invalid(Mseq[i]).argmax()
    rho_c_max.append(rho_c[imax])
    M_max.append(Mseq[i][imax])

    

fig, ax = plt.subplots()

for i in range(options.nseq)[::-1]:
    mycolor = color_cycle.next()
    ax.plot(rho_c, Mseq[i], color=mycolor)
    ax.plot(rho_c_max[i], M_max[i], 'o', color=mycolor)

ax.plot(mass_shedding["rho_c"], mass_shedding["M"], 'k-')
ax.plot(rho_K_max, M_K_max, 'o', color='black')
ax.fill_between(mass_shedding["rho_c"], mass_shedding["M"], 1.5*df["M"].max(), color='grey', alpha=0.15)
ax.plot()

ax.set_ylim(1.0, 3.0)
    
ax.set_xlabel(r"$\rho_c\ [10^{15}\ {\rm g}\, {\rm cm}^{-3}]$")
ax.set_ylabel(r"$M\ [M_\odot]$")